In [0]:
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/Modulo de Seguimiento/')

import SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs,greatest
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

# Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

## Base de seguimiento

In [0]:
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim
from pyspark import StorageLevel

import warnings

# Omitir todas las advertencias
warnings.filterwarnings("ignore")

# Parámetros de guardado
vUsuario = 'T39290'
vP = 'Vjqgqg$9171#'
vNumReg = 10000000
vStorage = "adlscu1lhclbackp05"
vRutaEdvDestino = "bcp-edv-fabseg/T39290/Bases DWH/Base Applicant Hipotecario"
vContainer = vRutaEdvDestino.split("/", 1)[0]
vRutaEDV = vRutaEdvDestino[len(vContainer) : len(vRutaEdvDestino)]
vQueryOrigen = "SELECT * FROM S35105.HM_MTZ_ADM_HIPOTECARIO_DRIVER WHERE CODMES >= 201701"

try:
    # Lectura de la base de hipotecario
    base_app_veh = (
        spark.read.format("jdbc")
        .option("url", "jdbc:oracle:thin:@PRODREG.credito.bcp.com.pe:1521/BCPDW3")       
        .option("dbtable", "(SELECT ROWNUM as R, t.* FROM ("+vQueryOrigen+") t ) mitabla")
        .option("fetchSize",100000) 
        .option("partitionColumn", "R")
        .option("lowerBound", "1")
        .option("upperBound", vNumReg)
        .option("numPartitions", "50")
        .option("user", vUsuario)
        .option("password", vP)
        .option("driver", "oracle.jdbc.driver.OracleDriver")
        .option("oracle.jdbc.timezoneAsRegion", "false")
        .option("oracle.jdbc.defaultCharSet", "UTF-8")
        .load()
    )
    print(f"Conexión exitosa: consulta '{vQueryOrigen}' leida correctamente.")

    # Guardar el DataFrame en un contenedor de Azure Storage Account
    base_app_veh.write.format("delta").mode("overwrite").save("abfss://"+vContainer+"@"+vStorage+".dfs.core.windows.net"+vRutaEDV)

    spark.sql(
    '''
        DROP TABLE IF EXISTS catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_BASE_APP_HIP
    '''
    )
    spark.sql('''
        CREATE TABLE 
            catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_BASE_APP_HIP
        USING DELTA LOCATION 
            'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T39290/Bases DWH/Base Applicant Hipotecario'
    '''
    )

except Exception as e:
    print(f"Conexión fallida: {e}")

In [0]:
base_app_veh.agg(F.max("CODMES")).show()

## Bases Espejo

### Hip. Trad y Pyme

In [0]:
!pip install pyreadstat
import pyreadstat
import pandas as pd
import os

# Puntuaciones historicas
espejo_hist = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_MM_APPSCORE_HIP_TRD_202504')

folder_path = '/Workspace/Users/luisfdiaz@bcp.com.pe/Seguimiento/Applicant Hipotecario/Inputs/Base Espejo Hip. Trad - Pyme'
files = [f for f in os.listdir(folder_path) if f.endswith('.sas7bdat')]


dfs = []
for file in files:
    file_path = os.path.join(folder_path, file)
    df, meta = pyreadstat.read_sas7bdat(file_path)
    dfs.append(df)

# Unir todos los DataFrames
espejo1 = pd.concat(dfs, ignore_index=True)
espejo1 = espejo1.fillna('')
espejo = spark.createDataFrame(espejo1)

# Uniendo puntuaciones
espejo_total = espejo_hist.unionByName(espejo, allowMissingColumns=True)

espejo_total.createOrReplaceTempView("espejo")
spark.sql(
    '''
    CREATE OR REPLACE TABLE
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_mm_appscore_hip_trad
    USING DELTA LOCATION 
        'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T39290/Seguimiento/02_Applicant_Hipotecario/mm_appscpre_hip_trad'
    AS SELECT * FROM espejo
    '''
)

# Conteo registros
espejo_sql = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_mm_appscore_hip_trad') 
info = espejo_sql.filter("CODMES_SOLICITUD >= 202201").groupBy("CODMES_SOLICITUD").agg(F.count("*").alias("REGISTROS")).orderBy("CODMES_SOLICITUD")
print_res(info)

### Hip. Ahlo

In [0]:
import pyreadstat
import pandas as pd
import os

# Puntuaciones historicas
espejo_hist = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_MM_APPSCORE_HIP_AHL_202504')

folder_path = '/Workspace/Users/luisfdiaz@bcp.com.pe/Seguimiento/Applicant Hipotecario/Inputs/Base Espejo Hip. Ahlo'
files = [f for f in os.listdir(folder_path) if f.endswith('.sas7bdat')]


dfs = []
for file in files:
    file_path = os.path.join(folder_path, file)
    df, meta = pyreadstat.read_sas7bdat(file_path)
    dfs.append(df)

# Unir todos los DataFrames
espejo1 = pd.concat(dfs, ignore_index=True)
espejo = spark.createDataFrame(espejo1)

# Uniendo puntuaciones
espejo_total = espejo_hist.unionByName(espejo, allowMissingColumns=True)

espejo_total.createOrReplaceTempView("espejo")
spark.sql(
    '''
    CREATE OR REPLACE TABLE
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_mm_appscore_hip_ahlo
    USING DELTA LOCATION 
        'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T39290/Seguimiento/02_Applicant_Hipotecario/mm_appscore_hip_ahlo'
    AS SELECT * FROM espejo
    '''
)

# Conteo registros
espejo_sql = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_mm_appscore_hip_ahlo') 
info = espejo_sql.filter("CODMES_SOLICITUD >= 202201").groupBy("CODMES_SOLICITUD").agg(F.count("*").alias("REGISTROS")).orderBy("CODMES_SOLICITUD")
print_res(info)

### Gahi

In [0]:
import pyreadstat
import pandas as pd
import os

base_espejo_ant=spark.sql("select distinct * from catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_MM_APPSCORE_GARANTIAHIP_202503")
base_espejo_m=spark.sql("select distinct * from catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_MM_APPSCORE_GARANTIAHIP_202504")
espejo_hist = base_espejo_ant.unionByName(base_espejo_m)

folder_path = '/Workspace/Users/luisfdiaz@bcp.com.pe/Seguimiento/Applicant Hipotecario/Inputs/Base Espejo GAHI'
files = [f for f in os.listdir(folder_path) if f.endswith('.sas7bdat')]


dfs = []
for file in files:
    file_path = os.path.join(folder_path, file)
    df, meta = pyreadstat.read_sas7bdat(file_path)
    dfs.append(df)

# Unir todos los DataFrames
espejo1 = pd.concat(dfs, ignore_index=True)
espejo = spark.createDataFrame(espejo1)

# Uniendo puntuaciones
espejo_total = espejo_hist.unionByName(espejo, allowMissingColumns=True)

espejo_total.createOrReplaceTempView("espejo")
spark.sql(
    '''
    CREATE OR REPLACE TABLE
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_mm_appscore_gahi
    USING DELTA LOCATION 
        'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T39290/Seguimiento/02_Applicant_Hipotecario/mm_appscore_gahi'
    AS SELECT * FROM espejo
    '''
)

# Conteo registros
espejo_sql = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_mm_appscore_gahi') 
info = espejo_sql.filter("CODMES_SOLICITUD >= 202201").groupBy("CODMES_SOLICITUD").agg(F.count("*").alias("REGISTROS")).orderBy("CODMES_SOLICITUD")
print_res(info)

## Puntuaciones

In [0]:
!pip install pyreadstat

### Hip. Trad y Pyme

In [0]:
import pyreadstat
import pandas as pd
import os

# Puntuaciones historicas
espejo_hist = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_ADM_HIP_TRADPYME_2Q24_202504')

folder_path = '/Workspace/Users/luisfdiaz@bcp.com.pe/Seguimiento/Applicant Hipotecario/Inputs/Puntuación Táctica Hip. Trad - Pyme'
files = [f for f in os.listdir(folder_path) if f.endswith('.sas7bdat')]

dfs = []
for file in files:
    file_path = os.path.join(folder_path, file)
    df, meta = pyreadstat.read_sas7bdat(file_path)
    dfs.append(df)

# Unir todos los DataFrames
espejo1 = pd.concat(dfs, ignore_index=True)
espejo = spark.createDataFrame(espejo1)

# Uniendo puntuaciones
espejo_total = espejo_hist.unionByName(espejo, allowMissingColumns=True)

espejo_total.createOrReplaceTempView("espejo")
spark.sql(
    '''
    CREATE OR REPLACE TABLE
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_hm_adm_hip_trad_score
    USING DELTA LOCATION 
        'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T39290/Seguimiento/02_Applicant_Hipotecario/hm_adm_hip_trad_score'
    AS SELECT * FROM espejo
    '''
)

# Conteo registros
espejo_sql = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_hm_adm_hip_trad_score') 
info = espejo_sql.filter("CODMES >= 202201").groupBy("CODMES").agg(F.count("*").alias("REGISTROS")).orderBy("CODMES")
print_res(info)

### Hip. Ahlo

In [0]:
!pip install pyreadstat

import pyreadstat
import pandas as pd
import os

# Puntuaciones historicas
espejo_hist = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_ADM_HIP_AHLO_Q424_202504')

folder_path = '/Workspace/Users/luisfdiaz@bcp.com.pe/Seguimiento/Applicant Hipotecario/Inputs/Puntuación Táctica Hip. Ahlo'
files = [f for f in os.listdir(folder_path) if f.endswith('.sas7bdat')]

dfs = []
for file in files:
    file_path = os.path.join(folder_path, file)
    df, meta = pyreadstat.read_sas7bdat(file_path)
    dfs.append(df)

# Unir todos los DataFrames
espejo1 = pd.concat(dfs, ignore_index=True)
espejo1 = espejo1.fillna('')
espejo = spark.createDataFrame(espejo1)

# Uniendo puntuaciones
espejo_total = espejo_hist.unionByName(espejo, allowMissingColumns=True)

espejo_total.createOrReplaceTempView("espejo")
spark.sql(
    '''
    CREATE OR REPLACE TABLE
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_hm_adm_hip_ahlo_score
    USING DELTA LOCATION 
        'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T39290/Seguimiento/02_Applicant_Hipotecario/hm_adm_hip_ahlo_score'
    AS SELECT * FROM espejo
    '''
)

# Conteo registros
espejo_sql = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_hm_adm_hip_ahlo_score') 
info = espejo_sql.filter("CODMES >= 202201").groupBy("CODMES").agg(F.count("*").alias("REGISTROS"), F.avg('PD_FWL_2024Q4').alias('PD')).orderBy("CODMES")
print_res(info)